In [61]:
import os
import ast
import json
import shutil
from PIL import Image
import pandas as pd
import numpy as np
from lib.datasets.utils import load_synds_list, load_deep_gestalt_encodings

 python train.py --outdir=training-runs --data=..\..\data\GestaltMatcherDB\GMDB_syndrome --gpus=1
 --cfg=auto --cond=1 --metrics=none  --resume=training-runs\00016-GMDB_syndrome-cond-auto1\training-runs\00016-GMDB_syndrome-cond-auto1 --snap=25


python train.py --outdir=training-runs --data=..\..\data\GAN\ --gpus=1 --cfg=auto --cond=1 --met
rics=none  --snap=25 --mirror=1 --aug=ada --dry-run --augpipe=gmdb


In [66]:
#df = pd.read_csv('./gmdb_metadata/gmdb_syndromes_v1.tsv', sep='\t')
df = pd.read_csv('../data/GestaltMatcherDB/v1.0.1/gmdb_metadata_v1.0.1/gmdb_syndromes_v1.0.1.csv', sep=',')
synd_ids = df.syndrome_id.values

# Prepare Training images

In [47]:
#df = pd.read_csv('./gmdb_metadata/gmdb_syndromes_v1.tsv', sep='\t')
df = pd.read_csv('../data/GestaltMatcherDB/v1.0.1/gmdb_metadata_v1.0.1/gmdb_syndromes_v1.0.1.csv', sep=',')
synd_ids = df.syndrome_id.values
work_path = '../synthesis/stylegan2-ada-pytorch/'
input_path = os.path.join('../data/GestaltMatcherDB/v1.0.1/gmdb_crops_v1.0.1/')
output_path = '../data/GAN/GMDB_syndrome_original_images_v1.0.1'
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [77]:
df['image_ids'] = df.apply(lambda x: [int(i.strip(' ').strip('\n')) for i in x['image_ids'][1:-1].split(' ') if i != ''], axis=1)

In [79]:
output_path = '../data/GAN/GMDB_syndrome_crops_v1.0.1'
input_path = '../data/GestaltMatcherDB/v1.0.1/gmdb_crops_v1.0.1/'
for idx, row in df.iterrows():
    synd_id = row['syndrome_id']
    image_ids = row['image_ids']
    synd_output_path = os.path.join(output_path, str(synd_id))
    if not os.path.exists(synd_output_path):
        os.makedirs(synd_output_path)
    for input_image_id in image_ids:
        input_file = os.path.join(input_path, '{}_crop_square.jpg'.format(input_image_id))
        output_file = os.path.join(synd_output_path, '{}.jpg'.format(input_image_id))
        shutil.copyfile(input_file, output_file)

 python dataset_tool.py --source=..\..\data\GAN\GMDB_syndrome_original_images_v1.0.1 --dest=..\..
\data\GAN\train_synd_images_v1.0.1 --width=128 --height=128


In [63]:
df = pd.read_csv('../data/GestaltMatcherDB/v1.0.1/gmdb_metadata_v1.0.1/gmdb_frequent_gallery_images_v1.0.1.csv', sep=',')
image_ids = df.image_id.values
output_prefix = '../data/GAN/GMDB_syndrome_crops_v1.0.1'
input_prefix = '../data/GestaltMatcherDB/v1.0.1/gmdb_crops_v1.0.1/'
if not os.path.exists(output_prefix):
    os.mkdir(output_prefix)
all_labels = []
count = 0
count_str = 0
for idx, row in df.iterrows():
    image_id = row['image_id']
    label = str(row['label'])
    input_image = os.path.join(input_prefix, '{}_crop_square.jpg'.format(image_id))
    if count == 1000:
        count_str += 1
        count = 0
    count_str_dir = '0'*(5-len(str(count_str))) + str(count_str)
    out_dir = os.path.join(output_prefix, count_str_dir)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)

    img = Image.open(input_image).convert('L')
    img = img.resize((128, 128))
    filename = 'img' + '0'*(8-len(str(image_id))) + str(image_id) + '.png'
    output_image = os.path.join(out_dir, filename)
    img.save(output_image, format='png')
    
    all_labels.append([count_str_dir + '/' + filename, int(label)])
    count += 1

In [64]:
json_string = {'labels': all_labels}
with open(output_prefix+ '\dataset.json', 'w') as outfile:
    json.dump(json_string, outfile)

# Save into syndrome folder

# Generate images from StyleGan

In [71]:
df = pd.read_csv('../data/GestaltMatcherDB/v1.0.1/gmdb_metadata_v1.0.1/gmdb_syndromes_v1.0.1.csv', sep=',')
synd_ids = df.syndrome_id.values
synd_num_of_images = {row['syndrome_id']: row['num_of_images'] for idx, row in df.iterrows()}

In [72]:
work_path = '../synthesis/stylegan2-ada-pytorch/'
#model_path = os.path.join(work_path, 'training-runs/00016-GMDB_syndrome-cond-auto1/network-snapshot-000140.pkl')
model_path = os.path.join(work_path, 'training-runs/00022-GMDB_syndrome_crops_v1.0-cond-mirror-auto1-ada-gmdb/network-snapshot-000300.pkl')
output_path = os.path.join(work_path, 'GMDB_train_synd_v1.0.1_gmdb_ada')
py_file = os.path.join(work_path, 'generate.py')
if not os.path.exists(output_path):
    os.makedirs(output_path)

count = 0
for idx, synd_id in enumerate(synd_ids):
    num_of_images = synd_num_of_images[idx]
    synd_output_path = os.path.join(output_path, str(synd_id))
    if not os.path.exists(synd_output_path):
        os.makedirs(synd_output_path)
    seeds = '{}-{}'.format(count, count+num_of_images-1)
    count = count+num_of_images
    cmd = ' '.join(['python', py_file, '--class={}'.format(idx), '--network={}'.format(model_path),
                   '--outdir={}'.format(synd_output_path), '--seeds={}'.format(seeds)])
    os.system(cmd)

In [21]:
print(cmd)

python ../synthesis/stylegan2-ada-pytorch/generate.py --class=0 --network=../synthesis/stylegan2-ada-pytorch/training-runs/00016-GMDB_syndrome-cond-auto1/network-snapshot-000120.pkl --outdir=../synthesis/stylegan2-ada-pytorch/GMDB_train_synd\0 --seeds=0-243


# Crop and encode

In [25]:
work_path = '../synthesis/stylegan2-ada-pytorch/'
input_path = os.path.join(work_path, 'GMDB_train_synd')
output_path = '../data/GMDB_gan_original_images'
if not os.path.exists(output_path):
    os.makedirs(output_path)

for idx, synd_id in enumerate(synd_ids):
    synd_input_path = os.path.join(input_path, str(synd_id))
    input_images = os.listdir(synd_input_path)
    for input_image in input_images:
        input_file = os.path.join(synd_input_path, input_image)
        image_id = int(input_image[4:-4])
        output_file = os.path.join(output_path, '{}.png'.format(image_id))
        shutil.copyfile(input_file, output_file)

In [26]:
train_df = pd.read_csv('gmdb_metadata/gmdb_train_images_v1.csv')
val_df = pd.read_csv('gmdb_metadata/gmdb_val_images_v1.csv')
test_df = pd.read_csv('gmdb_metadata/gmdb_test_images_v1.csv')
synd_train_dict = {}
synd_val_dict = {}
synd_test_dict = {}
for i, j  in train_df['label'].value_counts().items():
    synd_train_dict[i] = j

for i, j  in val_df['label'].value_counts().items():
    synd_val_dict[i] = j

for i, j  in test_df['label'].value_counts().items():
    synd_test_dict[i] = j

In [44]:
train_image_ids = np.array([])
train_labels = np.array([])
val_image_ids = np.array([])
val_labels = np.array([])
test_image_ids = np.array([])
test_labels = np.array([])
count = 0
synd_list = list(synd_train_dict.keys())
synd_list.sort()
for idx, synd_id in enumerate(synd_list):
    num_train_images = synd_train_dict[synd_id]
    num_val_images = synd_val_dict[synd_id]
    num_test_images = synd_test_dict[synd_id]
    train_image_ids = np.append(train_image_ids, np.arange(count, count+num_train_images))
    train_labels = np.append(train_labels, np.array([synd_id]*num_train_images))
    count += num_train_images
    val_image_ids = np.append(val_image_ids, np.arange(count, count+num_val_images))
    val_labels = np.append(val_labels, np.array([synd_id]*num_val_images))
    count += num_val_images
    test_image_ids = np.append(test_image_ids, np.arange(count, count+num_test_images))
    test_labels = np.append(test_labels, np.array([synd_id]*num_test_images))
    count += num_test_images

In [48]:
train_out_df = pd.DataFrame({'image_id': train_image_ids, 'label': train_labels, 'subject': train_image_ids})
train_out_df.to_csv('../data/gmdb_gan_train_images_v1.csv', index=False)
val_out_df = pd.DataFrame({'image_id': val_image_ids, 'label': val_labels, 'subject': val_image_ids})
val_out_df.to_csv('../data/gmdb_gan_val_images_v1.csv', index=False)
test_out_df = pd.DataFrame({'image_id': test_image_ids, 'label': test_labels, 'subject': test_image_ids})
val_out_df.to_csv('../data/gmdb_gan_test_images_v1.csv', index=False)

# Train model

In [101]:
train_df = pd.read_csv('gmdb_metadata/gmdb_train_images_v1.csv')
val_df = pd.read_csv('gmdb_metadata/gmdb_val_images_v1.csv')
test_df = pd.read_csv('gmdb_metadata/gmdb_test_images_v1.csv')
labels = test_df.label.values
synd_list = load_synds_list('lookup_table.txt')

embeddings = []
image_ids = []
control_subject_ids = []
for image_id in test_df.image_id.values:
    encoding = load_deep_gestalt_encodings('gmdb_gan_dg_encodings\{}.csv'.format(image_id), synd_list)
    embeddings.append(encoding['image_softmax_ranks'][str(image_id)])
    image_ids.append(image_id)

In [102]:
image_ranks = []
for i in range(len(image_ids)):
    ranks = embeddings[i]
    #print(labels[i])
    image_ranks.append(np.where(ranks == labels[i])[0][0])
image_ranks = np.array(image_ranks)

In [103]:
for i in [1, 5, 10, 30]:
    acc = sum(image_ranks < i)/len(image_ranks)*100
    print('Top-{} acc: {:.2f}%'.format(i, acc))


Top-1 acc: 14.17%
Top-5 acc: 29.17%
Top-10 acc: 37.78%
Top-30 acc: 54.17%


In [104]:
#val_out_df.to_csv('../data/gmdb_gan_test_images_v1.csv', index=False)
test_df = pd.read_csv('../data/gmdb_gan_test_images_v1.csv').astype('int')
labels = test_df.label.values
synd_list = load_synds_list('lookup_table.txt')

embeddings = []
image_ids = []
control_subject_ids = []
for image_id in test_df.image_id.values:
    encoding = load_deep_gestalt_encodings('gan_dg_encodings\{}.csv'.format(image_id), synd_list)
    embeddings.append(encoding['image_softmax_ranks'][str(image_id)])
    image_ids.append(image_id)
image_ranks = []
for i in range(len(image_ids)):
    ranks = embeddings[i]
    #print(labels[i])
    image_ranks.append(np.where(ranks == labels[i])[0][0])
image_ranks = np.array(image_ranks)
for i in [1, 5, 10, 30]:
    acc = sum(image_ranks < i)/len(image_ranks)*100
    print('Top-{} acc: {:.2f}%'.format(i, acc))


Top-1 acc: 55.42%
Top-5 acc: 77.09%
Top-10 acc: 83.74%
Top-30 acc: 93.60%


In [105]:
train_df = pd.read_csv('gmdb_metadata/gmdb_train_images_v1.csv')
val_df = pd.read_csv('gmdb_metadata/gmdb_val_images_v1.csv')
test_df = pd.read_csv('gmdb_metadata/gmdb_test_images_v1.csv')
labels = test_df.label.values
synd_list = load_synds_list('lookup_table.txt')

embeddings = []
image_ids = []
control_subject_ids = []
for image_id in test_df.image_id.values:
    encoding = load_deep_gestalt_encodings('gmdb_dg_encodings\{}.csv'.format(image_id), synd_list)
    embeddings.append(encoding['image_softmax_ranks'][str(image_id)])
    image_ids.append(image_id)
image_ranks = []
for i in range(len(image_ids)):
    ranks = embeddings[i]
    #print(labels[i])
    image_ranks.append(np.where(ranks == labels[i])[0][0])
image_ranks = np.array(image_ranks)
for i in [1, 5, 10, 30]:
    acc = sum(image_ranks < i)/len(image_ranks)*100
    print('Top-{} acc: {:.2f}%'.format(i, acc))

Top-1 acc: 37.22%
Top-5 acc: 59.72%
Top-10 acc: 69.44%
Top-30 acc: 84.17%


In [107]:
#val_out_df.to_csv('../data/gmdb_gan_test_images_v1.csv', index=False)
test_df = pd.read_csv('../data/gmdb_gan_test_images_v1.csv').astype('int')
labels = test_df.label.values
synd_list = load_synds_list('lookup_table.txt')

embeddings = []
image_ids = []
control_subject_ids = []
for image_id in test_df.image_id.values:
    encoding = load_deep_gestalt_encodings('gan_images_dg_encodings\{}.csv'.format(image_id), synd_list)
    embeddings.append(encoding['image_softmax_ranks'][str(image_id)])
    image_ids.append(image_id)
image_ranks = []
for i in range(len(image_ids)):
    ranks = embeddings[i]
    #print(labels[i])
    image_ranks.append(np.where(ranks == labels[i])[0][0])
image_ranks = np.array(image_ranks)
for i in [1, 5, 10, 30]:
    acc = sum(image_ranks < i)/len(image_ranks)*100
    print('Top-{} acc: {:.2f}%'.format(i, acc))

Top-1 acc: 19.95%
Top-5 acc: 34.98%
Top-10 acc: 47.04%
Top-30 acc: 64.78%
